In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

from sklearn.utils import class_weight, shuffle
from sklearn.model_selection import train_test_split


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 48
SEED = 1020

In [ ]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED)

In [ ]:
df_train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
df_test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')

x = df_train['id_code']
y = df_train['diagnosis']

x, y = shuffle(x, y, random_state=SEED)

In [ ]:
y.hist()

## Data Pre-processing

In [ ]:
def crop_image_from_gray(img,tol=5):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

def circle_crop(img, sigmaX=10, gaussian=True):
    img = crop_image_from_gray(img)
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    if gaussian: img=cv2.addWeighted(img, 4, cv2.GaussianBlur(img, (0,0), sigmaX), -4, 128)
    return img

In [ ]:
fig = plt.figure(figsize=(25, 25))
for class_id in sorted(y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(1, random_state=SEED).iterrows()):
        path=f"../input/aptos2019-blindness-detection/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        ax = fig.add_subplot(5, 5, i * 5 + class_id + 1, xticks=[], yticks=[])
        image_normal = circle_crop(image, gaussian=False)
        image_normal = cv2.resize(image_normal, (IMG_SIZE, IMG_SIZE))
        plt.imshow(image_normal)
        ax.set_title('Normal, ID: %s' % (row['id_code']) )
        
        ax = fig.add_subplot(5, 5, (i+1) * 5 + class_id + 1, xticks=[], yticks=[])
        img_circle_gaussian=circle_crop(image, gaussian=False)
        img_circle_gaussian=cv2.resize(img_circle_gaussian, (IMG_SIZE, IMG_SIZE))
        img_circle_gaussian=circle_crop(img_circle_gaussian, sigmaX=5, gaussian=True)
        img_circle_gaussian=circle_crop(img_circle_gaussian, gaussian=False)
        plt.imshow(img_circle_gaussian)
        ax.set_title('Circle Crop Gaussian Blur, ID: %s' % (row['id_code']) )
        
        ax = fig.add_subplot(5, 5, (i+2) * 5 + class_id + 1, xticks=[], yticks=[])
        img_circle_gaussian=circle_crop(image, gaussian=False)
        img_circle_gaussian=cv2.resize(img_circle_gaussian, (IMG_SIZE, IMG_SIZE))
        img_circle_gaussian=circle_crop(img_circle_gaussian, sigmaX=10, gaussian=True)
        img_circle_gaussian=circle_crop(img_circle_gaussian, gaussian=False)
        plt.imshow(img_circle_gaussian)
        ax.set_title('Circle Crop Gaussian Blur, ID: %s' % (row['id_code']) )
        
        ax = fig.add_subplot(5, 5, (i+3) * 5 + class_id + 1, xticks=[], yticks=[])
        img_circle_gaussian=circle_crop(image, gaussian=False)
        img_circle_gaussian=cv2.resize(img_circle_gaussian, (IMG_SIZE, IMG_SIZE))
        img_circle_gaussian=circle_crop(img_circle_gaussian, sigmaX=20, gaussian=True)
        img_circle_gaussian=circle_crop(img_circle_gaussian, gaussian=False)
        plt.imshow(img_circle_gaussian)
        ax.set_title('Circle Crop Gaussian Blur, ID: %s' % (row['id_code']) )
        
        ax = fig.add_subplot(5, 5, (i+4) * 5 + class_id + 1, xticks=[], yticks=[])
        img_circle_gaussian=circle_crop(image, gaussian=False)
        img_circle_gaussian=cv2.resize(img_circle_gaussian, (IMG_SIZE, IMG_SIZE))
        img_circle_gaussian=circle_crop(img_circle_gaussian, sigmaX=30, gaussian=True)
        img_circle_gaussian=circle_crop(img_circle_gaussian, gaussian=False)
        plt.imshow(img_circle_gaussian)
        ax.set_title('Circle Crop Gaussian Blur, ID: %s' % (row['id_code']) )
        
plt.savefig('./pre-processing_1.png')

In [ ]:
# import multiprocessing as mp
# NCORE = 20

# def process(q, iolock):
#     while True:
#         stuff = q.get()
#         if stuff is None:
#             break
#         idx, row, sets = stuff
#         path=f"../input/aptos2019-blindness-detection/{sets}_images/{row['id_code']}.png"
#         image = cv2.imread(path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
#         img_circle_gaussian=circle_crop(image, gaussian=False)
#         img_circle_gaussian=cv2.resize(img_circle_gaussian, (IMG_SIZE, IMG_SIZE))
#         img_circle_gaussian=circle_crop(img_circle_gaussian, sigmaX=5, gaussian=True)
#         img_circle_gaussian=circle_crop(img_circle_gaussian, gaussian=False)
        
#         if sets=='train':
#             cv2.imwrite(f"./{sets}/{row['diagnosis']}/{row['id_code']}.png", img_circle_gaussian)
#         else:
#             cv2.imwrite(f"./{sets}/{row['id_code']}.png", img_circle_gaussian)


# for class_id in sorted(y.unique()):
#     if not os.path.exists(f'./train/{class_id}/'): os.makedirs(f'./train/{class_id}/')
# if not os.path.exists(f'./test/'): os.makedirs(f'./test/')
    
# q = mp.Queue(maxsize=NCORE)
# iolock = mp.Lock()
# pool = mp.Pool(NCORE, initializer=process, initargs=(q, iolock))
# for i, (idx,row) in enumerate(df_train.iterrows()):
#     stuff = (idx, row, 'train')
#     q.put(stuff)
# for i, (idx,row) in enumerate(df_test.iterrows()):
#     stuff = (idx, row, 'test')
#     q.put(stuff)
# for _ in range(NCORE):
#     q.put(None)
# pool.close()
# pool.join()

In [ ]:
# df_train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
# df_test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
# submission = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
# df_train.to_csv("./train.csv")
# df_test.to_csv("./test.csv")
# submission.to_csv("./sample_submission.csv")

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.applications import *

In [ ]:
# input_size=(IMG_SIZE,IMG_SIZE,3)
# base_model_1 = ResNet50V2(input_shape=input_size, weights='imagenet', include_top=False)
# base_model_2 = InceptionV3(input_shape=input_size, weights='imagenet', include_top=False)
# base_model_3 = DenseNet201(input_shape=input_size, weights='imagenet', include_top=False)

# if not os.path.exists(f'./model/'): os.makedirs(f'./model/')
# base_model_1.save_weights(f'./model/resnet50v2.h5')
# base_model_2.save_weights(f'./model/inceptionv3.h5')
# base_model_3.save_weights(f'./model/densenet201.h5')

In [ ]:
def my_model_2(input_size=(IMG_SIZE,IMG_SIZE,3), cross_stitch=True):
    inp = keras.Input(shape=input_size)
    x = layers.Rescaling(1./255)(inp)
    
    base_model_1 = ResNet50V2(input_shape=input_size, 
                              weights=f'../input/aptos2019-512-tfstyle/model/resnet50v2.h5', 
                              include_top=False)
    base_model_1.trainable = False
    base_model_2 = InceptionV3(input_shape=input_size, 
                               weights=f'../input/aptos2019-512-tfstyle/model/inceptionv3.h5', 
                               include_top=False)
    base_model_2.trainable = False
    base_model_3 = DenseNet201(input_shape=input_size, 
                               weights=f'../input/aptos2019-512-tfstyle/model/densenet201.h5',
                               include_top=False)
    base_model_3.trainable = False 

    x_1 = base_model_1(x, training=False)
    x_2 = base_model_2(x, training=False)
    x_3 = base_model_3(x, training=False)
    x_1 = keras.layers.GlobalAveragePooling2D()(x_1)
    x_2 = keras.layers.GlobalAveragePooling2D()(x_2)
    x_3 = keras.layers.GlobalAveragePooling2D()(x_3)
                                                
    x_1 = keras.layers.Dense(800, activation='relu', kernel_initializer='lecun_normal', kernel_regularizer= keras.regularizers.l2(0.001))(x_1)
    x_2 = keras.layers.Dense(800, activation='relu', kernel_initializer='lecun_normal', kernel_regularizer= keras.regularizers.l2(0.001))(x_2)
    x_3 = keras.layers.Dense(800, activation='relu', kernel_initializer='lecun_normal', kernel_regularizer= keras.regularizers.l2(0.001))(x_3)

    cross_stitch_2 = tf.Variable([[0.5,0.5,0.5],[0.5,0.5,0.5],[0.5,0.5,0.5]], dtype=tf.float32, trainable=True)
    if cross_stitch==True:
        cross_x_1 = cross_stitch_2[0,0] * x_1 + cross_stitch_2[0,1] * x_2 + cross_stitch_2[0,2] * x_3
        cross_x_2 = cross_stitch_2[1,0] * x_1 + cross_stitch_2[1,1] * x_2 + cross_stitch_2[1,2] * x_3
        cross_x_3 = cross_stitch_2[2,0] * x_1 + cross_stitch_2[2,1] * x_2 + cross_stitch_2[2,2] * x_3
    else:
        cross_x_1 = x_1
        cross_x_2 = x_2
        cross_x_3 = x_3
    x_1 = keras.layers.Dense(200, activation='relu', kernel_initializer='lecun_normal', kernel_regularizer= keras.regularizers.l2(0.001))(cross_x_1)
    x_2 = keras.layers.Dense(200, activation='relu', kernel_initializer='lecun_normal', kernel_regularizer= keras.regularizers.l2(0.001))(cross_x_2)
    x_3 = keras.layers.Dense(200, activation='relu', kernel_initializer='lecun_normal', kernel_regularizer= keras.regularizers.l2(0.001))(cross_x_3)

    cross_stitch_3 = tf.Variable([[0.5,0.5,0.5],[0.5,0.5,0.5],[0.5,0.5,0.5]], dtype=tf.float32, trainable=True)
    if cross_stitch==True:
        cross_x_1 = cross_stitch_3[0,0] * x_1 + cross_stitch_3[0,1] * x_2 + cross_stitch_3[0,2] * x_3
        cross_x_2 = cross_stitch_3[1,0] * x_1 + cross_stitch_3[1,1] * x_2 + cross_stitch_3[1,2] * x_3
        cross_x_3 = cross_stitch_3[2,0] * x_1 + cross_stitch_3[2,1] * x_2 + cross_stitch_3[2,2] * x_3
    else:
        cross_x_1 = x_1
        cross_x_2 = x_2
        cross_x_3 = x_3
    x_1 = keras.layers.Dense(5)(cross_x_1)
    x_2 = keras.layers.Dense(5)(cross_x_2)
    x_3 = keras.layers.Dense(5)(cross_x_3)

    aver_vairable = tf.Variable([0.33333,0.33333,0.33333], dtype=tf.float32, trainable=False)
    x = aver_vairable[0] * x_1 + aver_vairable[1] * x_2 + aver_vairable[2] * x_3
    model = keras.Model(inputs=inp, outputs=x)
    return model

In [ ]:
model = my_model_2()
model.summary()

In [ ]:
data_aug = keras.Sequential(
  [
    layers.RandomFlip(input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.RandomRotation(0.4, fill_mode='constant', fill_value=0),
    layers.RandomZoom(0.1, fill_mode='constant', fill_value=0)
  ]
)

In [ ]:

train_ds = (
        tf.keras.utils.image_dataset_from_directory(
        '../input/aptos2019-512-tfstyle/train',
        validation_split=0.1,
        seed=1020,
        subset="training",
        image_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)
        .map(lambda x, y: (data_aug(x), y), num_parallel_calls=tf.data.AUTOTUNE)
        )
val_ds = (tf.keras.utils.image_dataset_from_directory(
        '../input/aptos2019-512-tfstyle/train',
        validation_split=0.1,
        seed=1020,
        subset="validation",
        image_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)
        )

In [ ]:
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs=20
history = model.fit(train_ds,validation_data=val_ds,epochs=epochs)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.ylim((0.5,1.5))
plt.show()

In [ ]:
test_data = pd.read_csv("../input/aptos2019-blindness-detection/test.csv")
test_data["filename"] = test_data["id_code"].map(lambda x:x+".png")
test_data.head()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

test_gen = ImageDataGenerator()
test_generator = test_gen.flow_from_dataframe(  
        dataframe=test_data,
        directory = "../input/aptos2019-blindness-detection/test_images",    
        x_col="filename",
        y_col=None,
        target_size = (IMG_SIZE,IMG_SIZE),
        batch_size = 16,
        shuffle = False,
        class_mode = None
        )

In [ ]:
predictions = model.predict_generator(test_generator, steps = len(test_generator.filenames))

In [ ]:
filenames=test_generator.filenames
results=pd.DataFrame({"id_code":filenames,
                      "diagnosis":np.argmax(predictions,axis=1)})
results['id_code'] = results['id_code'].map(lambda x: str(x)[:-4])
results.to_csv("submission.csv",index=False)

In [ ]:
results.diagnosis.hist()